# Large MNL work

Sam Maurer, April 2018 | Python 3.6

This notebook is for development, testing, and demonstration of the template for MNL with large numbers of alternatives.

In [2]:
import os; os.chdir('../')

In [3]:
import numpy as np
import pandas as pd

In [4]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca

/Users/maurer/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [6]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['household_id', 'taz', 'serialno', 'puma5', 'income', 'persons', 'hht', 'unittype', 'noc', 'bldgsz', 'tenure', 'vehicl', 'hinccat1', 'hinccat2', 'hhagecat', 'hsizecat', 'hfamily', 'hunittype', 'hnoccat', 'hwrkrcat', 'h0004', 'h0511', 'h1215', 'h1617', 'h1824', 'h2534', 'h3549', 'h5064', 'h6579', 'h80up', 'hworkers', 'hwork_f', 'hwork_p', 'huniv', 'hnwork', 'hretire', 'hpresch', 'hschpred', 'hschdriv', 'htypdwel', 'hownrent', 'hadnwst', 'hadwpst', 'hadkids', 'bucketbin', 'originalpuma', 'hmultiunit', 'building_id']

BUILDINGS
['building_id', 'parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id']



# Large MNL

In [7]:
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings']
m.choice_column = 'building_id'
m.alt_sample_size = 10
m.chooser_filters = ['household_id % 1000 < 1']

m.model_expression = 'res_price_per_sqft - 1'

m.name = 'large-mnl-test'
m.tags = ['sam', 'testing']

In [8]:
len(m._get_df(tables=m.choosers, filters=m.chooser_filters))

2608

In [9]:
%%time
m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:               
Model:         Multinomial Logit   Df Residuals:                   
Method:       Maximum Likelihood   Df Model:                       
Date:                              Pseudo R-squ.:                  
Time:                              Pseudo R-bar-squ.:              
AIC:                               Log-Likelihood:       -5,973.144
BIC:                               LL-Null:              -5,995.932
                        coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------
res_price_per_sqft    0.0005     0.000     9.968                       
CPU times: user 753 ms, sys: 445 ms, total: 1.2 s
Wall time: 1.17 s


In [10]:
m.fitted_parameters

[0.00045990926356891706]

In [11]:
m.register()

In [12]:
mm.get_step('large-mnl-test').fitted_parameters

[0.00045990926356891706]

In [13]:
m.out_chooser_filters = ['household_id % 1000 == 17']

In [14]:
%%time
m.run()

None


ValueError: cannot label index with a null key

In [ ]:
m.choices